In [5]:
import pandas as pd
import json


def load_jsonl(file_path):
    with open(file_path, 'rb') as f:
        law_docs_iterator = json.load(f)
        return law_docs_iterator



def legal_corpus_to_dataframe():
    """
    Chuyển đổi corpus pháp lý thành DataFrame.
    """
    read_legal_corpus = load_jsonl('data/legal_corpus.json')
    legal_df = pd.json_normalize(
        read_legal_corpus,
        meta=['id', 'law_id'],
        record_path='content'
    )
    return legal_df


df_legal_corpus = legal_corpus_to_dataframe()
print(df_legal_corpus.head())
df_train =  pd.DataFrame(load_jsonl('data/train.json'))
print(df_train.head())




   aid                                    content_Article id           law_id
0    0  1. Thông tư này quy định mã số, tiêu chuẩn chu...  0  14/2022/TT-NHNN
1    1  1. Kiểm soát viên cao cấp ngân hàng Mã số: 07....  0  14/2022/TT-NHNN
2    2  1. Có bản lĩnh chính trị vững vàng, kiên định ...  0  14/2022/TT-NHNN
3    3  1. Chức trách Là công chức có trình độ chuyên ...  0  14/2022/TT-NHNN
4    4  1. Chức trách Là công chức có trình độ chuyên ...  0  14/2022/TT-NHNN
     qid                                           question  \
0    933  Thưa luật sư tôi có đăng ký kết hôn trên pháp ...   
1   2997  Ai có quyền điều hành hoạt động của liên hiệp ...   
2  12282  Trình tự đăng ký hành nghề dịch vụ kế toán đượ...   
3    340  Thời hạn giải quyết hồ sơ nhận con nuôi là bao...   
4   7418  Hoạt động chuyển giao công nghệ trong cơ sở gi...   

           relevant_laws  
0  [53877, 53875, 53929]  
1                [24221]  
2  [29515, 29512, 58071]  
3                 [3543]  
4                [

## Hàm Tiền Xử Lý Văn Bản Chung (`preprocess_text`)

Hàm này thực hiện các bước tiền xử lý văn bản như sau:

1. **Chuyển về chữ thường:** Đồng nhất các từ để tránh phân biệt hoa/thường.
2. **Xử lý ký tự xuống dòng:** Thay thế ký tự `\n` bằng khoảng trắng để nối các dòng lại với nhau.
3. **Chuẩn hóa khoảng trắng:** Loại bỏ các khoảng trắng thừa, chỉ giữ lại một khoảng trắng giữa các từ.
4. **Tách từ (Word Segmentation):** Sử dụng `underthesea.word_tokenize` để tách từ tiếng Việt. Các từ ghép sẽ được nối bằng dấu gạch dưới (ví dụ: `kinh_doanh`).
5. **Loại bỏ các ký tự không phải chữ cái, số, hoặc dấu gạch dưới:** Giữ lại các từ đã tách và các số, loại bỏ các ký tự đặc biệt khác.
6. **Thêm dấu cách cho dấu câu:** Đảm bảo dấu câu được tách riêng, không dính vào từ.


In [6]:
import regex as re
from underthesea import word_tokenize

def preprocess_text(text):
    """
    Tiền xử lý văn bản:
    - Chuyển về chữ thường
    - Xử lý ký tự xuống dòng
    - Chuẩn hóa khoảng trắng
    - Tách từ tiếng Việt
    - Loại bỏ ký tự đặc biệt không cần thiết
    """
    if not isinstance(text, str):
        return ""

    # 1. Chuyển về chữ thường
    text = text.lower()

    # 2. Xử lý ký tự xuống dòng và chuẩn hóa khoảng trắng
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # 3. Thêm khoảng trắng quanh dấu câu để đảm bảo tách từ đúng
    # (Giữ lại các dấu cơ bản, loại bỏ các ký tự đặc biệt khác)
    text = re.sub(r'([.,!?;:()])', r' \1 ', text)
    text = re.sub(r'\s+', ' ', text).strip()    

    # 4. Tách từ bằng underthesea
    # `word_tokenize` trả về danh sách các từ đã tách
    tokens = word_tokenize(text, format="text") # format="text" trả về chuỗi đã nối gạch dưới

    # 5. Loại bỏ các ký tự không phải chữ cái, số, hoặc dấu gạch dưới
    # Giữ lại các từ đã tách và các số, bỏ các ký tự đặc biệt khác
    text_processed = re.sub(r'[^\p{L}\p{N}_ ]+', '', tokens, flags=re.UNICODE)
    text_processed = re.sub(r'\s+', ' ', text_processed).strip()

    return text_processed


**Tiền Xử Lý Dữ Liệu Huấn Luyện (train_data.json)**

In [25]:
import dask.dataframe as dd


df_train = dd.from_pandas(df_train, npartitions=4)
df_train['pre_question'] = df_train['question'].apply(
   lambda x: preprocess_text(x)
)
print(df_train.head())


     qid                                           question  \
0    933  Thưa luật sư tôi có đăng ký kết hôn trên pháp ...   
1   2997  Ai có quyền điều hành hoạt động của liên hiệp ...   
2  12282  Trình tự đăng ký hành nghề dịch vụ kế toán đượ...   
3    340  Thời hạn giải quyết hồ sơ nhận con nuôi là bao...   
4   7418  Hoạt động chuyển giao công nghệ trong cơ sở gi...   

           relevant_laws                                       pre_question  
0  [53877, 53875, 53929]  thưa luật_sư tôi có đăng_ký kết_hôn trên pháp_...  
1                [24221]  ai có quyền điều_hành hoạt_động của liên_hiệp ...  
2  [29515, 29512, 58071]  trình_tự đăng_ký hành_nghề dịch_vụ kế_toán đượ...  
3                 [3543]  thời_hạn giải_quyết hồ_sơ nhận con_nuôi là bao...  
4                [44548]  hoạt_động chuyển_giao công_nghệ trong cơ_sở gi...  


/Users/kat/KAT/TTNT/NLP/nlp-drill-chatbot/venv/lib/python3.11/site-packages/dask/dataframe/dask_expr/_collection.py:4399: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('question', 'object'))

  warnings.warn(meta_warning(meta))


**Tiền Xử Lý Kho Văn Bản Pháp Luật (legal_corpus.json) và Xây Dựng Cấu Trúc Tra Cứu**

In [31]:


df_legal_articles = dd.from_pandas(df_legal_corpus, npartitions=4)
df_legal_articles['pre_content'] = df_legal_articles['content_Article'].apply(
   lambda x: preprocess_text(x)
)
print(df_legal_articles.tail())



/Users/kat/KAT/TTNT/NLP/nlp-drill-chatbot/venv/lib/python3.11/site-packages/dask/dataframe/dask_expr/_collection.py:4399: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('content_Article', 'object'))

  warnings.warn(meta_warning(meta))


         aid                                    content_Article    id  \
59631  59631  Các Bộ trưởng, Thủ trưởng cơ quan ngang bộ, Th...  2154   
59632  59632  1. Sửa\r\nđổi, bổ sung khoản 2 Điều 3\nnhư sau...  2155   
59633  59633  1. Bộ trưởng Bộ Công Thương chịu trách nhiệm t...  2155   
59634  59634  1. Nghị định này có hiệu lực thi hành từ ngày ...  2155   
59635  59635  1. Sửa đổi\nkhoản 3 Điều 9 như sau: “3. Học ph...  2156   

              law_id                                        pre_content  
59631  72/2019/NĐ-CP  các bộ_trưởng thủ_trưởng cơ_quan ngang bộ thủ_...  
59632  18/2023/NĐ-CP  1 sửa_đổi bổ_sung khoản 2 điều 3 như sau 2 doa...  
59633  18/2023/NĐ-CP  1 bộ_trưởng bộ công_thương chịu trách_nhiệm tổ...  
59634  18/2023/NĐ-CP  1 nghị_định này có hiệu_lực thi_hành từ ngày 2...  
59635  97/2023/NĐ-CP  1 sửa_đổi khoản 3 điều 9 như sau 3 học_phí từ ...  



## 1. Kiến trúc (Architecture)
    - Encoder A (Query Encoder): Chuyên trách biến đổi các câu truy vấn (query) hoặc câu hỏi thành một vector số (embedding).

    - Encoder B (Document/Passage Encoder): Chuyên trách biến đổi các tài liệu (document), đoạn văn bản (passage), hoặc điều luật thành một vector số (embedding).

## 2. Giai đoạn Huấn luyện (Training Phase)**
    - Mục tiêu của huấn luyện Bi-encoder là điều chỉnh các trọng số của hai encoder (thực chất là một encoder duy nhất chia sẻ trọng số) sao cho:

    - Vector nhúng của một câu hỏi liên quan gần với vector nhúng của tài liệu trả lời nó.

    - Vector nhúng của một câu hỏi không liên quan xa với vector nhúng của tài liệu không liên quan.

    - Quá trình này thường sử dụng kỹ thuật Học tương phản (Contrastive Learning).
**Các bước huấn luyện:**
- **Đầu vào huấn luyện: Mô hình nhận vào các bộ ba (triplets) hoặc cặp (pairs):**

    - (Query Q, Positive Document D+): Một câu hỏi và một tài liệu thực sự liên quan đến câu hỏi đó.

    - (Query Q, Negative Document D-): Một câu hỏi và một tài liệu không liên quan đến câu hỏi đó. Tài liệu tiêu cực có thể được lấy mẫu ngẫu nhiên hoặc thông qua các chiến lược lấy mẫu "hard negative" (ví dụ: lấy tài liệu mà mô hình ban đầu nhầm lẫn là liên quan).

- **Quá trình mã hóa:**

    - Câu hỏi Q đi qua Encoder A để tạo ra Q_emb.

    - Tài liệu tích cực D+ đi qua Encoder B để tạo ra D_+emb.

    - Tài liệu tiêu cực D- đi qua Encoder B để tạo ra D_−emb.

- **Tính toán Độ tương đồng:**
    - Mô hình tính toán độ tương đồng giữa Q_emb và D_+emb (Positive Similarity).

    - Mô hình tính toán độ tương đồng giữa Q_emb và D_−emb (Negative Similarity).

    - Phép tính độ tương đồng phổ biến nhất là độ tương đồng cosin (cosine similarity) hoặc tích vô hướng (dot product).

- **Hàm mất mát (Loss Function) - Ví dụ: Multiple Negative Ranking Loss (MNRL):**

    - MNRL (được dùng trong thư viện sentence-transformers) là một loại Contrastive Loss hiệu quả.

    - Trong một batch huấn luyện, nó xem xét mỗi câu hỏi và tài liệu tích cực tương ứng.

    - Nó coi tất cả các tài liệu khác trong cùng batch (không phải là tài liệu tích cực của câu hỏi hiện tại) là các tài liệu tiêu cực tiềm năng.

    - Hàm mất mát sẽ cố gắng tối đa hóa điểm số của cặp (Q, D+) và tối thiểu hóa điểm số của cặp (Q, D-) trong cùng batch. Mục tiêu là làm cho Positive Similarity cao hơn Negative Similarity một khoảng an toàn (margin).

    - Về cơ bản, nó muốn xác suất của cặp (Q, D+) cao hơn các cặp (Q, D_khác).

- **Cập nhật trọng số:**
    - Dựa trên giá trị của hàm mất mát, thuật toán tối ưu hóa (ví dụ: AdamW) sẽ điều chỉnh trọng số của các encoder thông qua lan truyền ngược (backpropagation) để cải thiện khả năng phân biệt giữa các cặp liên quan và không liên quan.

## 3. Giai đoạn Sử dụng / Suy luận (Inference/Deployment Phase)
Sau khi Bi-encoder được huấn luyện xong, nó đã học được cách tạo ra các vector nhúng có ý nghĩa. Bây giờ chúng ta sử dụng nó để tìm kiếm:

- **Mã hóa Corpus Tài liệu (Offline Pre-computation):**

    - Sử dụng Encoder B để mã hóa tất cả các tài liệu (ví dụ: hàng ngàn, hàng triệu điều luật hoặc chunks điều luật) thành các vector nhúng.

    - Quá trình này chỉ cần thực hiện một lần và có thể mất nhiều thời gian nếu corpus lớn.

    - Các vector nhúng này được lưu trữ trong một chỉ mục tìm kiếm hiệu quả như FAISS. FAISS được tối ưu hóa để tìm kiếm láng giềng gần nhất (Nearest Neighbor Search) trong không gian vector đa chiều một cách rất nhanh chóng.

- **Xử lý Câu hỏi mới (Online Query):**
    - Khi một người dùng nhập một câu hỏi mới, câu hỏi đó sẽ được đưa qua Encoder A để tạo ra vector nhúng của câu hỏi đó (Q_new_emb).

    - Vector Q_new_emb này sau đó được sử dụng để truy vấn chỉ mục FAISS.

    - FAISS sẽ nhanh chóng tìm và trả về K vector tài liệu gần nhất với Q_new_emb trong không gian vector.

    - FAISS cũng trả về chỉ mục của các vector này, cho phép chúng ta tra cứu lại các tài liệu gốc tương ứng.


## GIAI ĐOẠN HUẤN LUYỆN:

    Câu hỏi Q   ------------> [Encoder A] -----> Q_emb
                                                 |
                                                 | (So sánh độ tương đồng và tính Loss)
                                                 |
    Tài liệu D+ ------------> [Encoder B] -----> D+_emb
    Tài liệu D- ------------> [Encoder B] -----> D-_emb
                                                    |
                                                    V
                                            Hàm mất mát (MNRL/Triplet)
                                            (Điều chỉnh trọng số Encoder A & B)

--------------------------------------------------------------------------------

## GIAI ĐOẠN SỬ DỤNG (Tìm kiếm):

    1. Mã hóa Corpus (làm 1 lần):
    Tài liệu D1 ---------> [Encoder B] -----> D1_emb
    Tài liệu D2 ---------> [Encoder B] -----> D2_emb
    ...
    Tài liệu DN ---------> [Encoder B] -----> DN_emb
                            (Tất cả D_emb được lưu vào FAISS Index)

    2. Xử lý Query mới (mỗi lần có câu hỏi):
    Câu hỏi Q_new ------> [Encoder A] -----> Q_new_emb
                                                    |
                                                    V
                                                [FAISS Index] (Tìm kiếm các D_emb gần nhất)
                                                    |
                                                    V
                                        Các Tài liệu liên quan nhất

In [1]:
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sentence_transformers import SentenceTransformer, InputExample, losses
from tqdm.auto import tqdm

class BiEncoderDatasetFromDF(Dataset):
    def __init__(self, df_train_data: pd.DataFrame, df_corpus_data: pd.DataFrame):
        self.df_train = df_train_data
        self.df_corpus = df_corpus_data
        
        self.data_samples = []
        
        # Iterate through each question in df_train
        for index, row in tqdm(self.df_train.iterrows(), total=len(self.df_train), desc="Chuẩn bị cặp huấn luyện"):
            question_text = row['pre_question'] 
            relevant_aids = row['relevant_laws']
            
            for aid_in_relevant_laws in relevant_aids:
                query_result = self.df_corpus.query(f"aid == {aid_in_relevant_laws}")
                
                positive_chunk_content = None
                if not query_result.empty:
                    # Lấy nội dung từ cột 'pre_content' và phần tử đầu tiên của mảng values
                    positive_chunk_content = query_result['pre_content'].values[0]
                
                if positive_chunk_content is not None:
                    self.data_samples.append(InputExample(texts=[question_text, positive_chunk_content]))
                # else:
                    # print(f"Cảnh báo: AID {aid_in_relevant_laws} từ relevant_laws không tìm thấy trong df_legal_articles.")

        print(f"Đã tạo {len(self.data_samples)} cặp huấn luyện (câu hỏi, chunk điều luật tích cực).")

    def __len__(self):
        return len(self.data_samples)

    def __getitem__(self, idx):
        return self.data_samples[idx]

/Users/kat/KAT/TTNT/NLP/nlp-drill-chatbot/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
